# Gun Detection with VGG16

This notebook demonstrates how to detecting guns appearing in images or video, using the pre-trained VGG16 deep learning model.

To get started, you must have the following requirements installed:

1. tensorflow - to preprocess data and work with the VGG16 model
2. numpy - to prepare data during testing
3. matplotlib - to display images in the Jupyter notebook
4. notebook - to run this notebook in a Jupyter server

It is recommended to have a virtual environment to isolate these requirements from the rest of your system. This can be done using Python's virtualenv package.

First, open a Terminal (Command Prompt on Windows) in the same folder as this notebook and create a virtual environment:

```shell
python3 -m venv env
```

Next, activate the virtual environment. For Windows users:

```shell
.\env\Scripts\activate
```

For Linux and MacOS users:

```shell
source env/bin/activate
```

Now you can safely install the above requirements in your virtual environment:

```shell
pip install tensorflow numpy matplotlib notebook
```

After installation is complete, launch the Jupyter server to edit this notebook:

```shell
jupyter notebook
```

## Getting Started

The following imports are necessary to work with this notebook:

In [2]:
# General imports

import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Tensorflow and Keras imports

from tensorflow import keras
from keras.models import Model
from keras.utils import load_img
from keras.models import load_model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

Just like in the previous example, we will be performing **transfer learning** with the **VGG16** pre-trained model from **Keras Applications**.

## The Dataset

For this scenario, we will be using a custom dataset of gun images, derived from multiple sources on the Internet.

The images in the dataset have a **high level of variety and diversity**. This ensures that the model can generalize better to locate and identify guns when operationalized in a real-world scenario.

### Exploratory Data Analysis

First, we'll inspect how the dataset is structured.

In [3]:
!tree "./data" -d

./data
├── train
│   ├── gun
│   └── nogun
└── validation
    ├── gun
    └── no gun

6 directories


As shown above, the dataset has been divided into 2 sub-directories:

1. `train` - training data
2. `validation` - validation data (which can be used for testing the model)

We can also inspect how many images belong to each class in the training and validation data.

In [5]:
def count_images(directory):
    return sum(
        1 for _ in os.listdir(directory) if _.endswith((".png", ".jpg", ".jpeg"))
    )


def summarize_data(base_dir):
    return {
        split: {
            category: count_images(os.path.join(base_dir, split, category))
            for category in ["gun", "no-gun"]
        }
        for split in ["train", "validation"]
    }


base_dir = "./data"
data_summary = summarize_data(base_dir)

for split, categories in data_summary.items():
    print(f"{split.capitalize()} data:")
    for category, count in categories.items():
        print(f"  {category}: {count} images")
    print()

Train data:
  gun: 60 images
  no-gun: 50 images

Validation data:
  gun: 45 images
  no-gun: 50 images



Notice how we have a very limited number of images to work with in our dataset, since we built this dataset from scratch.

### Image Augmentation

When training deep learning models using images, it's always better to use lots of data, especially images with slight variations in them, which can help the model generalize better.

But since we have limited data available, we can apply **image augmentation** to expand our dataset.

Image augmentation is the process of applying different transformations on our existing images, which results in multiple transformed copies of the same image. Each copy differs from its original image through minor differences created using rotation, flipping, shift etc.

Keras allows us to do this using the `ImageDataGenerator` class. In our case, we can augment both the training and testing data with different augmentation criteria:

In [7]:
# Instantiate a data generator object to augment data
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

Next, we can load the images using the `flow_from_directory()` method in the data generator object.

In [8]:
# Load the training data via the data generator
train_generator = datagen.flow_from_directory(
    directory="./data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training",
)

# Load the validation data via the data generator
validation_generator = datagen.flow_from_directory(
    directory="./data/validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation",
)

Found 88 images belonging to 2 classes.
Found 19 images belonging to 2 classes.
